## Load python libraries

In [1]:
import configparser
import gzip
import os
import glob
import subprocess
import multiprocessing
import shutil
import pandas as pd

## Import information from configuration file

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

path_data = config.get('paths', 'path_data')

print(path_data)

../data


## Define functions

In [9]:
'''
Function to obtain paths to the directories where the fastq files are located.
Currently requires demultiplexed data with gzipped files.
'''
def fastq_directory_paths():
    fastq_dir = set()
    file_paths = glob.glob(path_data + '/**/*.fastq.gz', recursive=True)
    directory_paths = map(os.path.dirname, file_paths)
    fastq_dir.update(directory_paths)
    return sorted(fastq_dir)

'''
Function to obtain sample names that are used as base to track the samples through the analysis
'''
basename = lambda paths: [path.split('/')[-2] if path.endswith('/') else path.split('/')[-1] for path in paths]

'''
Functions to count the number of raw sequences per sample
'''
def count_sequences(file_path):
    count = 0
    with gzip.open(file_path, 'rt') as gz_file:
        for line in gz_file:
            if line.startswith('@'):
                count += 1
    return count

def accumulate_counts(folder_path):
    counts = {}
    for folder in folder_path:
        counts[folder] = 0  # Initialize the folder key with count 0
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            if file_name.endswith('.fastq.gz'):
                counts[folder] += count_sequences(file_path)
    return counts

'''
Wrapper function for NanoFilt
'''
def filtering(folder_list, minimum_length, maximum_length, qscore, base_list):
    for folder, base in zip(folder_list, base_list):
        if os.path.exists(os.path.join('results/qc', base)) and os.path.isdir(os.path.join('results/qc', base)):
            pass
        else:
            os.makedirs(os.path.join('results/qc', base))
        for file_name in os.listdir(folder):
            if file_name.endswith('.fastq.gz'):
                file_path = os.path.join(folder, file_name)
                file_name_unzipped = file_name[:-3]
                command = f'gunzip -c {file_path} | NanoFilt \
                          --length {minimum_length} \
                          --maxlength {maximum_length} \
                          -q {qscore} | gzip > ./results/qc/{base}/{file_name}'
                subprocess.run(command, shell=True)
                
'''
Function to concatenate filtered sequences to one file per sample
'''
def concatenate(bases):
    for base in bases:
        command = f'cat ./results/qc/{base}/*.fastq.gz > ./results/qc/{base}/{base}_concatenated.fastq.gz'
        subprocess.run(command, shell=True)
        
        command = f'gunzip ./results/qc/{base}/{base}_concatenated.fastq.gz'
        subprocess.run(command, shell=True)
        
        command = f'sed -n "1~4s/^@/>/p;2~4p" ./results/qc/{base}/{base}_concatenated.fastq > ./results/qc/{base}/{base}_concatenated.fasta'
        subprocess.run(command, shell=True)

        #command = f'gzip ./results/qc/{base}/{base}_concatenated.fasta'
        #subprocess.run(command, shell=True)
        
'''
Function to count the number of sequences per sample after filtering
'''
def count_sequences_concat(base_name):
    counts = {}
    for base in base_name:
        counts[base] = 0  # Initialize the count for each base
        file_path = './results/qc/' + base + '/' + base + '_concatenated.fasta'
        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('>'):
                    counts[base] += 1
    return counts

'''
Wrapper function to generate sequencing summeries using NanoStat and MultiQC
'''
def stats(base_name):
    os.chdir(wdir)
    os.makedirs('multiqc')
    for base in base_name:
        input_file = './results/qc/' + base + '/' + base + '_concatenated.fastq'
        output_dir = './multiqc'
        output_name = base + "_statreports"    
        print(input_file)
        print(output_dir)
        command = [
            "NanoStat",
            "--fastq",
            input_file,
            "--outdir", output_dir,
            "--name", output_name
        ]   
    
        subprocess.run(command)
        os.chdir(wdir)
    os.chdir('./multiqc')
    command = [
        "multiqc",
        "."
    ]
    subprocess.run(command)
    os.chdir(wdir)

'''
Function to convert sequence files in fasta format to csv
'''
def fasta2csv(base_name):
    for base in base_name:
        fasta = './results/qc/' + base + '/' + base + '_concatenated.fasta'
        output = './results/qc/' + base + '/' + base + '_concatenated.csv'

        out_lines = []
        temp_line = ''
        with open(fasta, 'r') as fp:
            for line in fp:
                if line.startswith('>'):
                    out_lines.append(temp_line)
                    temp_line = line.strip() + ','
                else:
                    temp_line += line.strip()
        out_lines.append(temp_line)

        with open(output, 'w') as fp_out:
            fp_out.write('id,sequence' + '\n'.join(out_lines))
            
'''
Wrapper function to run the ashure clustering algorithm
'''
def cluster(base):
    os.chdir(wdir)
    shutil.copy('./ashure.py', './results/qc/' + base)
    shutil.copy('./bilge_pype.py', './results/qc/' + base)
    os.chdir('./results/qc/' + base)
    script_path = "./ashure.py"
    input_file = base + "_concatenated.csv"
    output_file = base + "_clusters.csv"    
    
    command = [
        script_path,
        "clst",
        "-i", input_file,
        "-o", output_file,
        "-iter", config.get('ashure', 'niter'),
        "-r"
    ]   
    
    subprocess.run(command)
    os.chdir(wdir)

'''
Function to convert sequence files in csv format to fasta
'''
def csv2fasta(base_name):
    for base in base_name:
        csv_path = './results/qc/' + base + '/' + base + '_clusters.csv'
        output_path = './results/qc/' + base + '/' + base + '_clusters.fasta'

        if os.path.exists(csv_path):
            out_lines = []
            temp_line = ''
            with open(csv_path, 'r') as csv_file:
                for line in csv_file:
                    cols = line.split(",")
                    out_lines.append(temp_line)
                    temp_line = ">" + cols[0] + "\n" + cols[1] + "\n"

            out_lines.append(temp_line)

            with open(output_path, 'w') as csv_out:
                csv_out.write(''.join(out_lines)[13:])
                
'''
Wrapper function to run cutadapt
'''
def remove_primers(base_name):
    for base in base_name:
        file_path = './results/qc/' + base + '/' + base + '_clusters.fasta'
        out_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
        if os.path.exists(file_path):
            command = [
                'cutadapt',
                '-a', 'CAGCAGCCGCGGTAATTCC;max_error_rate=0.20',
                '-g', 'CCCGTGTTGAGTCAAATTAAGC;max_error_rate=0.20',
                '--revcomp',
                '-o', out_path,
                file_path
            ]
            subprocess.run(command)

'''
Wrapper function to run blastn
'''                        
def blast(base_name):
    for base in base_name:
        file_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
        db = config.get('BLAST', 'db')
        if os.path.exists(file_path):
            print("Running blastn on", base)
            output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'

            command = [
                "blastn",
                "-db", config.get('paths', 'path_to_blastdb'),
                "-query", file_path,
                "-task", "blastn",
                "-dust", "no",
                "-num_threads", str(config.get('BLAST', 'numthreads')),
                "-outfmt", "7 delim=, sseqid stitle qacc sacc evalue bitscore length pident",
                "-max_target_seqs", str(config.get('BLAST', 'mts')),
                "-perc_identity", str(config.get('BLAST', 'pct_ident')),
                "-out", output_csv
            ]

            subprocess.run(command)

'''
Function to handle the blastn output files and generate a concatenated table with the taxonomic annotations
'''
def make_output_file(base_name):
    db = config.get('BLAST', 'db')
    for base in base_name:
        input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
        output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
        if os.path.exists(input_csv):
            with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
                for line in infile:
                    if not line.startswith('#'):
                        outfile.write(line)
                    
    for base in base_name:
        input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
        print(input_csv)
        output_csv = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
        if os.path.exists(input_csv) and os.path.getsize(input_csv) > 0:
            # load file
            df = pd.read_csv(input_csv, sep=',')

            # add column names
            df.columns=['accession', 'taxonomic_annotation', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity']

            # select only rows with alignment length >= 500 bp
            df2 = df[df['alignment_length'] >= 500]

            # arrange rows by match percentage
            df3 = df2.sort_values(by=['percentage_identity'], ascending=False)

            # keep only first row of each ASV
            df4 = df3.drop_duplicates(subset=['cluster'], keep='first', inplace=False, ignore_index=False)

            # add sample name information
            df4['#sample_name'] = base

            df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')

            df5 = df4[['#sample_name', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity', 'taxonomic_annotation']]

            df5.to_csv(output_csv, sep=';', index=False, header=False)

    intermediate = '_' + db + '_eDNA.csv'
    final = db + '_eDNA.csv'
    if os.path.exists(intermediate):
        os.remove(intermediate)
    else:
        pass
    try:
        with open(intermediate, "w") as f:
            f.write("")
    except OSError as e:
        rint(f"Error creating file: {e}")
    for base in base_name:
        file_path = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
        if os.path.exists(file_path):
            with open(file_path, "r") as input_file, open(intermediate, "a") as output_file:
                output_file.write(input_file.read())

    with open(intermediate, "r") as input_file, open(final, "w") as output_file:
        output_file.write("counts,cluster,accession,accession,evalue,bitscore,alignment_length,percentage_identity,taxonomic_annotation\n")
        for line in input_file:
            if not line.startswith("#"):
                output_file.write(line.replace(";", ",").replace("|", ","))

## Actual execution of the workflow

In [10]:
# Load paths to directories where fastq files are located
fastq_dir = fastq_directory_paths()
print(fastq_dir)

# Extract sample names from data
base_name = basename(fastq_dir)
print(base_name)

# Count number of raw sequences per sample
print(accumulate_counts(fastq_dir))

# Filter sequences using NanoFilt
filtering(fastq_dir, config.get('NanoFilt', 'minlength'), config.get('NanoFilt', 'maxlength'), config.get('NanoFilt', 'qscore'), base_name)

# Concatenate filtered sequences to one file per sample
concatenate(base_name)
        
# Count number of sequences per sample after filtering
print(count_sequences_concat(base_name))

# Make sure you save the original working directory before using moving around directories
wdir = os.getcwd()
print(wdir)

# Run NanoStat
stats(base_name)

# Convert fasta files to csv
fasta2csv(base_name)

# Run the actual clustering
if __name__ == '__main__':
    num_processes = 8  # Number of available CPU cores 
    with multiprocessing.Pool(processes=num_processes) as pool:
        pool.map(cluster, base_name)

# Move back to original working directory after clustering
os.chdir(wdir)

# Convert csv files to fasta
csv2fasta(base_name)

# Remove primers using a wrapper function for cutadapt
remove_primers(base_name)

# Taxonomic annotation using blastn
blast(base_name)

# Handle the blastn output files and generate a concatenated table with the taxonomic annotations
make_output_file(base_name)

['../data/barcode01', '../data/barcode02', '../data/barcode03']
['barcode01', 'barcode02', 'barcode03']
{'../data/barcode01': 28513, '../data/barcode02': 28554, '../data/barcode03': 28469}
{'barcode01': 25040, 'barcode02': 26721, 'barcode03': 20929}
/home/pascal/Documents/git_projects/grifo/src_0.6
./results/qc/barcode01/barcode01_concatenated.fastq
./multiqc
./results/qc/barcode02/barcode02_concatenated.fastq
./multiqc
./results/qc/barcode03/barcode03_concatenated.fastq
./multiqc

/// ]8;id=208268;https://multiqc.info\MultiQC]8;;\ 🔍 v1.22.2

       file_search | Search path: /home/pascal/Documents/git_projects/grifo/src_0.6/multiqc
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 3/3    
          nanostat | Found 3 reports
     write_results | Data        : multiqc_data
     write_results | Report      : multiqc_report.html
           multiqc | MultiQC complete
pid[11984] 2024-06-05 21:28:49.016 INFO: check_toolchain: /home/pascal/anaconda3/envs/GEANS/bin/minima

Traceback (most recent call last):
  File "./ashure.py", line 1169, in <module>
    main()
  File "./ashure.py", line 1157, in main
    data, flist = perform_cluster(df, df_d, max_iter=config['clst_N_iter'], csize=config['clst_csize'], N=config['clst_N'], th_s=config['clst_th_s'], th_m=config['clst_th_m'], pw_config=config['clst_pw_config'], msa_config=config['msa_config'], workspace=config['clst_folder'], track_file=config['clst_iter_out'], timestamp=True)
  File "./ashure.py", line 561, in perform_cluster
    df_c = bpy.cluster_sample(df_q, N=5, th=0.8, rsize=1, config=pw_config, workspace=workspace)
  File "/home/pascal/Documents/git_projects/grifo/src_0.6/results/qc/barcode03/bilge_pype.py", line 1351, in cluster_sample
    df_c.append(df_q[df_q['id'].isin(qry[ridx[:rsize]])][['id','sequence']])
  File "/home/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/pandas/core/generic.py", line 5989, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'Data

./results/qc/barcode01/barcode01_SILVA_138_blastn2.csv
./results/qc/barcode02/barcode02_SILVA_138_blastn2.csv
./results/qc/barcode03/barcode03_SILVA_138_blastn2.csv


## Taxonomic assignment using PR2

In [35]:
path_to_blastdb = "/home/pascal/Documents/GEANS/eDNA_18S/PR2_5.0.0/pr2_version_5.0.0_SSU_taxo_long.fasta"
numthreads = 8
mts = 10
pct_ident = 90
db = 'PR2'

for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
    if os.path.exists(file_path):
        print("Running blastn on", base)
        output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'

        command = [
            "blastn",
            "-db", path_to_blastdb,
            "-query", file_path,
            "-task", "blastn",
            "-dust", "no",
            "-num_threads", str(numthreads),
            "-outfmt", "7 delim=, sseqid stitle qacc sacc evalue bitscore length pident",
            "-max_target_seqs", str(mts),
            "-perc_identity", str(pct_ident),
            "-out", output_csv
        ]

        subprocess.run(command)

Running blastn on barcode01


Running blastn on barcode02


Running blastn on barcode03


Running blastn on barcode04


Running blastn on barcode05


Running blastn on barcode06


Running blastn on barcode07


Running blastn on barcode08
Running blastn on barcode09
Running blastn on barcode10


Running blastn on barcode11
Running blastn on barcode12


In [36]:
db = 'PR2'


for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    if os.path.exists(input_csv):
        with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
            for line in infile:
                if not line.startswith('#'):
                    outfile.write(line)



In [37]:
db = 'PR2'

for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    if os.path.exists(input_csv):
        with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
            for line in infile:
                if not line.startswith('#'):
                    comma_count = line.count(',')
                    if comma_count == 10:
                        comma_indices = []
                        for i, char in enumerate(line):
                            if char == ',':
                                comma_indices.append(i)

                        if len(comma_indices) >= 6:
                            line = line[:comma_indices[0]] + line[comma_indices[0]+1:]
                            line = line[:comma_indices[2]-1] + line[comma_indices[2]:]
                            line = line[:comma_indices[5]-2] + line[comma_indices[5]-1:]
                    elif comma_count == 13:
                        comma_indices = []
                        for i, char in enumerate(line):
                            if char == ',':
                                comma_indices.append(i)

                        indices_to_remove = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # Create a list to track the indices to remove

                        if len(comma_indices) >= 13:
                            indices_to_remove[0] = 1  # Mark the first comma to remove
                            indices_to_remove[1] = 1  # Mark the second comma to remove
                            indices_to_remove[3] = 1  # Mark the fourth comma to remove
                            indices_to_remove[4] = 1  # Mark the fifth comma to remove
                            indices_to_remove[7] = 1  # Mark the eighth comma to remove
                            indices_to_remove[8] = 1  # Mark the ninth comma to remove

                        updated_line = ""
                        for i, char in enumerate(line):
                            if char == ',' and indices_to_remove[comma_indices.index(i)] == 1:
                                continue
                            updated_line += char

                        line = updated_line
                    outfile.write(line)

In [38]:

db = 'PR2'




for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    print(input_csv)
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
    if os.path.exists(input_csv) and os.path.getsize(input_csv) > 0:
        # load file
        df = pd.read_csv(input_csv, sep=',')
    
        # add column names
        df.columns=['accession', 'taxonomic_annotation', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity']

        # select only rows with alignment length >= 500 bp
        df2 = df[df['alignment_length'] >= 500]

        # arrange rows by match percentage
        df3 = df2.sort_values(by=['percentage_identity'], ascending=False)

        # keep only first row of each ASV
        df4 = df3.drop_duplicates(subset=['cluster'], keep='first', inplace=False, ignore_index=False)

        # add sample name information
        df4['#sample_name'] = base

        df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')

        df5 = df4[['#sample_name', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity', 'taxonomic_annotation']]


        df5.to_csv(output_csv, sep=';', index=False, header=False)


./results/qc/barcode01/barcode01_PR2_blastn2.csv
./results/qc/barcode02/barcode02_PR2_blastn2.csv
./results/qc/barcode03/barcode03_PR2_blastn2.csv
./results/qc/barcode04/barcode04_PR2_blastn2.csv
./results/qc/barcode05/barcode05_PR2_blastn2.csv
./results/qc/barcode06/barcode06_PR2_blastn2.csv
./results/qc/barcode07/barcode07_PR2_blastn2.csv
./results/qc/barcode08/barcode08_PR2_blastn2.csv
./results/qc/barcode09/barcode09_PR2_blastn2.csv
./results/qc/barcode10/barcode10_PR2_blastn2.csv
./results/qc/barcode11/barcode11_PR2_blastn2.csv
./results/qc/barcode12/barcode12_PR2_blastn2.csv


/tmp/ipykernel_1122442/2543769090.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['#sample_name'] = base
/tmp/ipykernel_1122442/2543769090.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')
/tmp/ipykernel_1122442/2543769090.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [39]:
if os.path.exists("_PR2_eDNA.csv"):
    os.remove("_PR2_eDNA.csv")

for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
    if os.path.exists(file_path):
        with open(file_path, "r") as input_file, open("_PR2_eDNA.csv", "a") as output_file:
            output_file.write(input_file.read())

with open("_PR2_eDNA.csv", "r") as input_file, open("PR2_eDNA.csv", "w") as output_file:
    output_file.write("counts,cluster,accession,accession,evalue,bitscore,alignment_length,percentage_identity,taxonomic_annotation\n")
    for line in input_file:
        if not line.startswith("#"):
            output_file.write(line.replace(";", ",").replace("|", ","))

#shutil.copy("PR2_eDNA.csv", os.path.join(current_dir, "PR2_eDNA.csv"))